In [1]:
import json
from collections import OrderedDict

def reorder_question_fields(input_file, output_file):
    """
    JSON 파일에서 question과 question_type의 순서를 바꾸는 함수
    
    Args:
        input_file (str): 입력 JSON 파일 경로
        output_file (str): 출력 JSON 파일 경로
    """
    try:
        # JSON 파일 읽기
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # 각 항목의 input 필드에서 question과 question_type 순서 변경
        for item in data:
            if 'input' in item and isinstance(item['input'], dict):
                # 기존 input 데이터 저장
                input_data = item['input']
                
                # OrderedDict를 사용하여 순서 제어
                new_input = OrderedDict()
                
                # question_type을 먼저 추가
                if 'question_type' in input_data:
                    new_input['question_type'] = input_data['question_type']
                
                # question을 나중에 추가
                if 'question' in input_data:
                    new_input['question'] = input_data['question']
                
                # 다른 필드들도 추가 (있다면)
                for key, value in input_data.items():
                    if key not in ['question', 'question_type']:
                        new_input[key] = value
                
                # 새로운 순서로 input 업데이트
                item['input'] = dict(new_input)
        
        # 변경된 데이터를 새 파일에 저장
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        
        print(f"필드 순서가 변경되었습니다: {input_file} -> {output_file}")
        print(f"총 {len(data)}개 항목이 처리되었습니다.")
        
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {input_file}")
    except json.JSONDecodeError:
        print(f"JSON 파일 형식이 올바르지 않습니다: {input_file}")
    except Exception as e:
        print(f"오류가 발생했습니다: {str(e)}")

def main():
    """
    메인 함수 - 현재 predictions.json 파일의 필드 순서를 변경
    """
    input_file = "predictions.json"
    output_file = "predictions_reordered.json"
    
    print("question과 question_type의 순서를 변경합니다...")
    print(f"입력 파일: {input_file}")
    print(f"출력 파일: {output_file}")
    print("-" * 50)
    
    reorder_question_fields(input_file, output_file)

if __name__ == "__main__":
    main()


question과 question_type의 순서를 변경합니다...
입력 파일: predictions.json
출력 파일: predictions_reordered.json
--------------------------------------------------
필드 순서가 변경되었습니다: predictions.json -> predictions_reordered.json
총 498개 항목이 처리되었습니다.


In [7]:
import json

def check_answer_endings(json_file):
    """
    JSON 파일에서 answer가 '다.'로 끝나지 않는 항목의 개수를 확인하는 함수
    
    Args:
        json_file (str): JSON 파일 경로
    """
    try:
        # JSON 파일 읽기
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        total_count = 0
        not_ending_with_da = 0
        items_not_ending_with_da = []
        
        # 각 항목 확인
        for item in data:
            if 'output' in item and isinstance(item['output'], dict):
                if 'answer' in item['output']:
                    answer = item['output']['answer']
                    total_count += 1
                    
                    # answer가 '다.'로 끝나지 않는 경우
                    if not answer.strip().endswith('다.'):
                        not_ending_with_da += 1
                        # 모든 데이터 저장
                        items_not_ending_with_da.append({
                            'id': item.get('id', 'N/A'),
                            'answer': answer.strip(),
                            'full_answer_length': len(answer.strip())
                        })
        
        # 결과 출력
        print(f"전체 answer 항목 수: {total_count}")
        print(f"'다.'로 끝나지 않는 answer 수: {not_ending_with_da}")
        print(f"비율: {not_ending_with_da/total_count*100:.2f}%" if total_count > 0 else "비율: 계산할 수 없음")
        print("-" * 50)
        
        if items_not_ending_with_da:
            print("'다.'로 끝나지 않는 모든 항목:")
            for i, item in enumerate(items_not_ending_with_da, 1):
                print(f"{i}. ID: {item['id']}")
                print(f"   전체 답변: {item['answer']}")
                print(f"   답변 길이: {item['full_answer_length']}자")
                print("-" * 40)
        
        return not_ending_with_da, total_count
        
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {json_file}")
        return 0, 0
    except json.JSONDecodeError:
        print(f"JSON 파일 형식이 올바르지 않습니다: {json_file}")
        return 0, 0
    except Exception as e:
        print(f"오류가 발생했습니다: {str(e)}")
        return 0, 0

# 실행
print("predictions_reordered.json 파일 분석 중...")
not_ending_count, total_count = check_answer_endings("predictions_reordered.json")

# 추가로 원본 파일도 확인
print("\n" + "="*60)
print("predictions.json 파일 분석 중...")  
not_ending_count_orig, total_count_orig = check_answer_endings("predictions.json")

predictions_reordered.json 파일 분석 중...
전체 answer 항목 수: 498
'다.'로 끝나지 않는 answer 수: 29
비율: 5.82%
--------------------------------------------------
'다.'로 끝나지 않는 모든 항목:
1. ID: 796
   전체 답변: "녀석은 탁자 위에 몸을 둥근 채 있었다."가 옳다. '뉘인'은 '뉘어진'의 줄임어이다. '뉘어진'은 '뉘어'가 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'
   답변 길이: 364자
----------------------------------------
2. ID: 819
   전체 답변: "계란프라이를 먹었다."가 옳다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)

In [8]:
import json
import re

def remove_repetitions(text):
    """
    텍스트에서 반복되는 부분을 탐지하고 제거하는 함수
    """
    # 1. 문장 단위 반복 제거 (마지막 문장이 반복되는 경우)
    sentences = text.split('.')
    if len(sentences) > 2:
        # 마지막 빈 문자열 제거
        if sentences[-1].strip() == '':
            sentences = sentences[:-1]
        
        # 마지막 문장과 그 이전 문장들 비교
        last_sentence = sentences[-1].strip()
        if last_sentence:
            for i in range(len(sentences) - 2, -1, -1):
                if sentences[i].strip() == last_sentence:
                    # 반복 발견, 마지막 문장 제거
                    sentences = sentences[:-1]
                    break
    
    # 2. 단어/구문 단위 반복 제거
    result = '.'.join(sentences)
    if result.endswith('.'):
        result = result[:-1]
    
    # 3. 연속된 같은 문자열 패턴 제거 (예: "abc abc abc" -> "abc")
    words = result.split()
    if len(words) > 1:
        # 마지막 단어부터 확인
        for i in range(len(words) - 1, 0, -1):
            current_word = words[i]
            if i > 0 and words[i-1] == current_word:
                # 연속된 같은 단어 발견
                words = words[:i]
                break
    
    result = ' '.join(words)
    
    # 4. 특정 패턴의 반복 제거 (예: "'단어' '단어'" -> "'단어'")
    # 작은따옴표로 둘러싸인 단어의 반복
    pattern = r"'([^']+)'\s+'(\1)'"
    result = re.sub(pattern, r"'\1'", result)
    
    return result.strip()

def fix_incomplete_answers(json_file, output_file):
    """
    '다.'로 끝나지 않는 답변에서 반복을 제거하고 수정하는 함수
    """
    try:
        # JSON 파일 읽기
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        fixed_count = 0
        total_incomplete = 0
        
        # 각 항목 확인 및 수정
        for item in data:
            if 'output' in item and isinstance(item['output'], dict):
                if 'answer' in item['output']:
                    answer = item['output']['answer'].strip()
                    
                    # '다.'로 끝나지 않는 경우
                    if not answer.endswith('다.'):
                        total_incomplete += 1
                        original_answer = answer
                        
                        # 반복 제거
                        fixed_answer = remove_repetitions(answer)
                        
                        # '다.'로 끝나도록 수정 시도
                        if fixed_answer and not fixed_answer.endswith('다.'):
                            # 일반적인 한국어 종결어미 패턴 확인
                            if fixed_answer.endswith(('습니다', '합니다', '입니다')):
                                fixed_answer += '.'
                            elif fixed_answer.endswith(('다', '요', '음', '네', '지')):
                                if not fixed_answer.endswith('다'):
                                    # 문맥에 따라 적절한 종결어미 추가
                                    fixed_answer += '다.'
                                else:
                                    fixed_answer += '.'
                            else:
                                # 마지막 문자가 완전하지 않은 경우 (예: 중간에 끊어진 경우)
                                # 마지막 완전한 문장까지만 유지
                                sentences = fixed_answer.split('.')
                                if len(sentences) > 1:
                                    # 마지막 불완전한 부분 제거
                                    complete_sentences = []
                                    for sentence in sentences[:-1]:
                                        if sentence.strip():
                                            complete_sentences.append(sentence.strip())
                                    if complete_sentences:
                                        fixed_answer = '. '.join(complete_sentences) + '.'
                                    else:
                                        fixed_answer = original_answer + '다.'
                                else:
                                    fixed_answer += '다.'
                        
                        # 수정된 경우에만 업데이트
                        if fixed_answer != original_answer:
                            item['output']['answer'] = fixed_answer
                            fixed_count += 1
                            print(f"ID {item.get('id', 'N/A')} 수정됨:")
                            print(f"  원본: {original_answer[:100]}...")
                            print(f"  수정: {fixed_answer[:100]}...")
                            print("-" * 50)
        
        # 수정된 데이터 저장
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        
        print(f"\n수정 완료!")
        print(f"전체 불완전한 답변: {total_incomplete}개")
        print(f"수정된 답변: {fixed_count}개")
        print(f"저장된 파일: {output_file}")
        
        return fixed_count, total_incomplete
        
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {json_file}")
        return 0, 0
    except json.JSONDecodeError:
        print(f"JSON 파일 형식이 올바르지 않습니다: {json_file}")
        return 0, 0
    except Exception as e:
        print(f"오류가 발생했습니다: {str(e)}")
        return 0, 0

# 실행
print("반복 제거 및 답변 수정 작업 시작...")
fixed_count, total_incomplete = fix_incomplete_answers("predictions.json", "predictions_fixed.json")

print("\n" + "="*60)
print("수정된 파일 검증 중...")
# 수정된 파일 검증
not_ending_count_fixed, total_count_fixed = check_answer_endings("predictions_fixed.json")

반복 제거 및 답변 수정 작업 시작...
ID 796 수정됨:
  원본: "녀석은 탁자 위에 몸을 둥근 채 있었다."가 옳다. '뉘인'은 '뉘어진'의 줄임어이다. '뉘어진'은 '뉘어'가 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '뉘어'가 아닌 '...
  수정: "녀석은 탁자 위에 몸을 둥근 채 있었다. "가 옳다. '뉘인'은 '뉘어진'의 줄임어이다....
--------------------------------------------------
ID 819 수정됨:
  원본: "계란프라이를 먹었다."가 옳다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프...
  수정: "계란프라이를 먹었다. "가 옳다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '프라이'로 표기한다. 프라이(fry)는 '...
--------------------------------------------------
ID 835 수정됨:
  원본: "이번 사업의 성공률은 상당히 높다."가 옳다. '율'은 '률'로만 쓰인다. '율'은 '률'로만 쓰고, '률'은 '율'로만 쓰인다. '율'은 '률'로만 쓰이고, '율'은 '율'로만...
  수정: "이번 사업의 성공률은 상당히 높다. "가 옳다. '율'은 '률'로만 쓰인다. '율'은 '률'로만 쓰고, '률'은 '율'로만 쓰인다. '율'은 '률'로만 쓰이고, '율'은 '율'로...
--------------------------------------------------
ID 876 수정됨:
  원본: "그는 우의정이라는 높은 벼슬에도 올랐다고 합니다."가 옳다. 'ㄷ' 소리로 나는 받침 중에서 'ㄷ'으로 적는 것은 소리가 나는 대로 적는 것이 아니라, 그음이 나는 경우에 한한다...
  수정: "그는 우의정이라는 높은 벼슬에도 올랐다고 합니다. "가 옳다. 'ㄷ' 소리로 나는 받침 

In [ ]:
import json
import re

def advanced_remove_repetitions(text):
    """
    텍스트에서 반복되는 부분을 더 강력하게 탐지하고 제거하는 함수
    """
    # 1. 전체 문장에서 반복되는 구문 패턴 찾기
    def find_and_remove_patterns(text):
        # 문장을 단어로 분할
        words = text.split()
        if len(words) < 4:
            return text
        
        # 연속된 반복 패턴 찾기
        cleaned_words = []
        i = 0
        
        while i < len(words):
            # 현재 단어부터 시작하는 반복 패턴 찾기
            pattern_found = False
            
            # 2단어부터 10단어까지의 패턴 확인
            for pattern_length in range(2, min(11, len(words) - i + 1)):
                if i + pattern_length * 2 <= len(words):
                    pattern = words[i:i + pattern_length]
                    next_pattern = words[i + pattern_length:i + pattern_length * 2]
                    
                    if pattern == next_pattern:
                        # 반복 패턴 발견
                        cleaned_words.extend(pattern)
                        
                        # 연속된 같은 패턴이 더 있는지 확인
                        skip_count = pattern_length * 2
                        while (i + skip_count + pattern_length <= len(words) and 
                               words[i + skip_count:i + skip_count + pattern_length] == pattern):
                            skip_count += pattern_length
                        
                        i += skip_count
                        pattern_found = True
                        break
            
            if not pattern_found:
                cleaned_words.append(words[i])
                i += 1
        
        return ' '.join(cleaned_words)
    
    # 2. 반복 패턴 제거 적용
    result = find_and_remove_patterns(text)
    
    # 3. 특정 반복 패턴들 정규식으로 제거
    patterns_to_remove = [
        # 따옴표 안의 내용이 반복되는 패턴
        r"'([^']+)'\s*'(\1)'\s*'(\1)'",  # 3번 반복
        r"'([^']+)'\s*'(\1)'",           # 2번 반복
    ]
    
    for pattern in patterns_to_remove:
        result = re.sub(pattern, r"'\1'", result)
    
    # 4. 문장 끝 정리
    result = re.sub(r'\s+', ' ', result)  # 여러 공백을 하나로
    result = re.sub(r'\s+([.!?])', r'\1', result)  # 문장부호 앞 공백 제거
    
    return result.strip()

def enhanced_fix_incomplete_answers(json_file, output_file):
    """
    향상된 반복 제거 및 답변 수정 함수
    """
    try:
        # JSON 파일 읽기
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        fixed_count = 0
        total_incomplete = 0
        
        # 각 항목 확인 및 수정
        for item in data:
            if 'output' in item and isinstance(item['output'], dict):
                if 'answer' in item['output']:
                    answer = item['output']['answer'].strip()
                    
                    # '다.'로 끝나지 않는 경우
                    if not answer.endswith('다.'):
                        total_incomplete += 1
                        original_answer = answer
                        
                        # 강화된 반복 제거
                        fixed_answer = advanced_remove_repetitions(answer)
                        
                        # '다.'로 끝나도록 수정
                        if fixed_answer and not fixed_answer.endswith('다.'):
                            # 마지막 완전한 문장까지만 유지
                            sentences = re.split(r'([.!?])', fixed_answer)
                            complete_parts = []
                            
                            # 마지막 완전한 문장 찾기
                            for i in range(len(sentences) - 1, -1, -1):
                                if sentences[i] in '.!?':
                                    complete_parts = sentences[:i+1]
                                    break
                            
                            if complete_parts:
                                fixed_answer = ''.join(complete_parts)
                            else:
                                # 완전한 문장이 없으면 적절한 종결어미 추가
                                if not fixed_answer.endswith('.'):
                                    if fixed_answer.endswith(('습니다', '합니다', '입니다')):
                                        fixed_answer += '.'
                                    else:
                                        fixed_answer += '다.'
                        
                        # 수정된 경우에만 업데이트
                        if len(fixed_answer) < len(original_answer):  # 실제로 단축된 경우만
                            item['output']['answer'] = fixed_answer
                            fixed_count += 1
                            print(f"ID {item.get('id', 'N/A')} 수정됨:")
                            print(f"  원본 길이: {len(original_answer)}자 -> 수정 길이: {len(fixed_answer)}자")
                            print(f"  단축률: {((len(original_answer) - len(fixed_answer)) / len(original_answer) * 100):.1f}%")
                            print(f"  수정 후: {fixed_answer}")
                            print("-" * 60)
        
        # 수정된 데이터 저장
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        
        print(f"\n✅ 향상된 수정 완료!")
        print(f"전체 불완전한 답변: {total_incomplete}개")
        print(f"실제 수정된 답변: {fixed_count}개")
        print(f"저장된 파일: {output_file}")
        
        return fixed_count, total_incomplete
        
    except Exception as e:
        print(f"오류가 발생했습니다: {str(e)}")
        return 0, 0

# 실행
print("🔧 향상된 반복 제거 및 답변 수정 작업 시작...")
fixed_count, total_incomplete = enhanced_fix_incomplete_answers("predictions.json", "predictions_enhanced_fixed.json")

print("\n" + "="*60)
print("📊 수정된 파일 검증 중...")
# 수정된 파일 검증
not_ending_count_enhanced, total_count_enhanced = check_answer_endings("predictions_enhanced_fixed.json")